In [1]:
# 95% CI calculation (PP fragmentation)
import pandas as pd
import numpy as np
import xlwings as xw
pd.set_option('display.float_format', '{:.2e}'.format)
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtl/PlasticFADE.xlsx"  # CHECK: confirm file path
sheet_name = "Uncertainty"
data_CI_left = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI_PP_frag = data_CI_left.iloc[36:42] # Row index minus 3, change this range for other polymers
print(data_CI_PP_frag)
a_i, delta_i, b_i, alpha_i, c_i, beta_i = data_CI_PP_frag.iloc[:, 2].values
a_i_std, delta_i_std, b_i_std, alpha_i_std, c_i_std, beta_i_std = data_CI_PP_frag.iloc[:, 3].values

data_input = pd.read_excel(file_path, sheet_name="Fig.4", usecols="R:U", skiprows=1)
input_PP_frag = data_input.iloc[0:8, :]  # Change for other polymers
print("\n", input_PP_frag)
input_PP_frag.columns = ['s', 'I_j', 'P_j', 'C_j']

# Monte Carlo setup
N = 10000
np.random.seed(42)
results = []
# Loop through each row of input
for index, row in input_PP_frag.iterrows():
    s = row['s']
    I_j = row['I_j']
    P_j = row['P_j']
    a_i_samples = np.random.lognormal(np.log(a_i), a_i_std, N)
    delta_i_samples = np.random.lognormal(np.log(delta_i), delta_i_std, N)
    b_i_samples = np.random.lognormal(np.log(b_i), b_i_std, N)
    alpha_i_samples = np.random.lognormal(np.log(alpha_i), alpha_i_std, N)
    c_i_samples = np.random.lognormal(np.log(c_i), c_i_std, N)
    beta_i_samples = np.random.lognormal(np.log(beta_i), beta_i_std, N)

    # Impose caps to filter out physically meaningless values (in most empirical contexts, exponents above 4–5 are rare)
    delta_i_samples = np.clip(delta_i_samples, 0, 5)
    alpha_i_samples = np.clip(alpha_i_samples, 0, 5)
    beta_i_samples = np.clip(beta_i_samples, 0, 5)

    k_samples = a_i_samples * (s**delta_i_samples) * (b_i_samples * I_j**alpha_i_samples + c_i_samples * P_j**beta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples
    if np.all(k_samples == 0):
        lower_bound, upper_bound = 0, 0
    else:
        log_k = np.log10(k_samples[k_samples > 0])  # Exclude zeros (for subsoil & sediment)
        lower_bound = 10 ** np.percentile(log_k, 2.5)
        upper_bound = 10 ** np.percentile(log_k, 97.5)
    k_point = a_i * (s**delta_i) * (b_i * I_j**alpha_i + c_i * P_j**beta_i)
    results.append({'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})
results_CI = pd.DataFrame(results)
print("\n", results_CI)

# Save to Excel
wb = xw.Book(file_path)
sheet = wb.sheets["Fig.4"]
start_row = 3  # Check: confirm row number
sheet.range(f'I{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'K{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)
wb.save()
wb.close()

             Process Parameter  Estimate Standard deviation
36  PP fragmentation       a_i  4.66e-06           1.85e-05
37               NaN   delta_i  2.26e+00           1.23e+00
38               NaN       b_i  5.34e-04           1.53e-03
39               NaN   alpha_i  5.00e-13           9.01e-01
40               NaN       c_i  8.68e+00           9.42e-08
41               NaN    beta_i  1.44e+00           3.67e-01

       SA:V  I_j [W/m2] P_j [mW] C_j [CFU/ml]
0 6.00e+01    1.00e+01 1.85e-05       250000
1 6.00e+01    1.00e-01 8.09e-07    670000000
2 6.00e+01    0.00e+00        0       482000
3 6.00e+01    1.00e+01 9.20e-03       250000
4 6.00e+01    0.00e+00 4.71e-06        38500
5 6.00e+01    0.00e+00        0       482000
6 1.81e+01    1.12e+01 2.78e-02            -
7 1.81e+01    0.00e+00 2.78e-02            -

    k_point  CI_lower  CI_upper
0 2.59e-05  6.12e-09  4.10e+00
1 2.59e-05  5.75e-09  2.06e+00
2 0.00e+00  0.00e+00  0.00e+00
3 5.26e-04  2.46e-08  4.84e+02
4 9.39e-09  8.17

In [3]:
# 95% CI calculation (PP degradation)
pd.set_option('display.float_format', '{:.2e}'.format)
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtl/PlasticFADE.xlsx"  # CHECK: confirm file path
sheet_name = "Uncertainty"
data_CI_left = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI_PP_degr = data_CI_left.iloc[42:48] # Row index minus 3, change this range for other polymers
print(data_CI_PP_degr)
x_i, tau_i, y_i, theta_i, z_i, eta_i = data_CI_PP_degr.iloc[:, 2].values
x_i_std, tau_i_std, y_i_std, theta_i_std, z_i_std, eta_i_std = data_CI_PP_degr.iloc[:, 3].values

data_input = pd.read_excel(file_path, sheet_name="Fig.4", usecols="R:U", skiprows=1)
input_PP_degr = data_input.iloc[8:15, :]  # Change for other polymers
print("\n", input_PP_degr)
input_PP_degr.columns = ['s', 'I_j', 'P_j', 'C_j']

# Monte Carlo setup
N = 10000
np.random.seed(42)
results = []
# Loop through each row of input
for index, row in input_PP_degr.iterrows():
    s = row['s']
    I_j = row['I_j']
    C_j = row['C_j']
    x_i_samples = np.random.lognormal(np.log(x_i), x_i_std, N)
    tau_i_samples = np.random.lognormal(np.log(tau_i), tau_i_std, N)
    y_i_samples = np.random.lognormal(np.log(y_i), y_i_std, N)
    theta_i_samples = np.random.lognormal(np.log(theta_i), theta_i_std, N)
    z_i_samples = np.random.lognormal(np.log(z_i), z_i_std, N)
    eta_i_samples = np.random.lognormal(np.log(eta_i), eta_i_std, N)
    k_samples = x_i_samples * (s**tau_i_samples) * (y_i_samples * I_j**theta_i_samples + z_i_samples * C_j**eta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples
    log_k = np.log10(k_samples)
    log_lower = np.percentile(log_k, 2.5)
    log_upper = np.percentile(log_k, 97.5)
    lower_bound = 10 ** log_lower
    upper_bound = 10 ** log_upper
    k_point = x_i * (s**tau_i) * (y_i * I_j**theta_i + z_i * C_j**eta_i)  # Point estimate of k_degr
    results.append({'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})
results_CI = pd.DataFrame(results)
print("\n", results_CI)

# Save to Excel
wb = xw.Book(file_path)
sheet = wb.sheets["Fig.4"]
start_row = 11  # Check: confirm row number
sheet.range(f'I{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'K{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)
wb.save()
wb.close()

           Process Parameter  Estimate Standard deviation
42  PP degradation       x_i  5.50e-03           5.10e-03
43             NaN     tau_i  1.37e-02           1.12e-01
44             NaN       y_i  1.72e-02           1.62e-03
45             NaN   theta_i  7.05e-01           3.80e-01
46             NaN       z_i  1.62e-05           3.81e-05
47             NaN     eta_i  4.42e-01           1.11e-01

        SA:V  I_j [W/m2] P_j [mW] C_j [CFU/ml]
8  6.00e+01    1.00e+01 1.85e-05       250000
9  6.00e+01    1.00e-01 8.09e-07    670000000
10 6.00e+01    0.00e+00        0       482000
11 6.00e+01    1.00e+01 9.20e-03       250000
12 6.00e+01    0.00e+00 4.71e-06        38500
13 6.00e+01    0.00e+00        0       482000
14 5.67e+01    1.00e+01        -            0

    k_point  CI_lower  CI_upper
0 5.32e-04  2.40e-04  3.13e-03
1 7.70e-04  1.46e-04  6.99e-03
2 3.06e-05  9.74e-06  1.24e-04
3 5.32e-04  2.42e-04  2.99e-03
4 1.00e-05  3.92e-06  3.09e-05
5 3.06e-05  1.00e-05  1.30e-04
6 5.0

In [5]:
# 95% CI calculation (PE degradation)
pd.set_option('display.float_format', '{:.2e}'.format)
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtl/PlasticFADE.xlsx"  # CHECK: confirm file path
sheet_name = "Uncertainty"
data_CI_left = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI_PE_degr = data_CI_left.iloc[30:36] # Row index minus 3, change this range for other polymers
print(data_CI_PE_degr)
x_i, tau_i, y_i, theta_i, z_i, eta_i = data_CI_PE_degr.iloc[:, 2].values
x_i_std, tau_i_std, y_i_std, theta_i_std, z_i_std, eta_i_std = data_CI_PE_degr.iloc[:, 3].values

data_input = pd.read_excel(file_path, sheet_name="Fig.4", usecols="R:U", skiprows=1)
input_PE_degr = data_input.iloc[15, :]  # Change for other polymers
print("\n", input_PE_degr)
input_PE_degr.index = ['s', 'I_j', 'P_j', 'C_j']

s = input_PE_degr['s']
I_j = input_PE_degr['I_j']
C_j = input_PE_degr['C_j']

# Monte Carlo simulations
N = 10000
np.random.seed(42)
x_i_samples = np.random.lognormal(np.log(x_i), x_i_std, N)
tau_i_samples = np.random.lognormal(np.log(tau_i), tau_i_std, N)
y_i_samples = np.random.lognormal(np.log(y_i), y_i_std, N)
theta_i_samples = np.random.lognormal(np.log(theta_i), theta_i_std, N)
z_i_samples = np.random.lognormal(np.log(z_i), z_i_std, N)
eta_i_samples = np.random.lognormal(np.log(eta_i), eta_i_std, N)

k_samples = x_i_samples * (s**tau_i_samples) * (y_i_samples * I_j**theta_i_samples + z_i_samples * C_j**eta_i_samples)
k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples
log_k = np.log10(k_samples)
log_lower = np.percentile(log_k, 2.5)
log_upper = np.percentile(log_k, 97.5)
lower_bound = 10 ** log_lower
upper_bound = 10 ** log_upper
k_point = x_i * (s**tau_i) * (y_i * I_j**theta_i + z_i * C_j**eta_i)

# Save to Excel
wb = xw.Book(file_path)
sheet = wb.sheets["Fig.4"]
start_row = 18  # Check: confirm row number
sheet.range(f'I{start_row}').value = lower_bound
sheet.range(f'K{start_row}').value = upper_bound
wb.save()
wb.close()

           Process Parameter  Estimate Standard deviation
30  PE degradation       x_i  4.72e-09           2.48e-09
31             NaN     tau_i  1.35e+00           1.93e-10
32             NaN       y_i  3.68e-04           1.92e-06
33             NaN   theta_i  4.99e+00           1.14e-09
34             NaN       z_i  5.33e-05           1.33e-05
35             NaN     eta_i  6.51e-01           1.46e-08

 SA:V           5.67e+01
I_j [W/m2]     1.00e+01
P_j [mW]              -
C_j [CFU/ml]          0
Name: 15, dtype: object
